In [ ]:
!pip install "gspread==6.1.3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.1.4
    Uninstalling gspread-6.1.4:
      Successfully uninstalled gspread-6.1.4


In [ ]:
sheets_link = "https://docs.google.com/spreadsheets/d/1pPddxC4NHOInTeO2IfNzHOHsLWVEGtshseqXhA2YJ8M/edit?gid=0#gid=0"

In [ ]:
import gspread
import pandas as pd

key = "GOOGLE_API_KEY"
gc = gspread.api_key(key)

sh = gc.open_by_key("1pPddxC4NHOInTeO2IfNzHOHsLWVEGtshseqXhA2YJ8M")

worksheet = sh.sheet1

data = worksheet.get_all_values()

df = pd.DataFrame(data[1:], columns=data[0])

In [ ]:
df_en = df.loc[df["language"].eq("en"), :]
df_en.head()

,id,subject,body,answer,type,queue,priority,language,business_type,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9
0,1001352387736,Urgent: Critical impact on enterprise network ...,"Dear Customer Support Team, We are experiencin...",Subject: Re: Urgent: Critical impact on enterp...,Incident,Technical Support,high,en,IT Services,Service Outage,Network Issue,Urgent Issue,Technical Support,Problem Resolution,Critical Failure,System Crash,Service Disruption,
1,1004699418379,Intermittent Cursor Freezing Issue on Dell XPS,"Dear Customer Support,<br><br>I hope this mess...","Dear <name>,\n\n\nThank you for reaching out r...",Incident,Product Support,low,en,Tech Online Store,Technical Support,Product Support,Hardware Failure,Problem Resolution,Urgent Issue,Service Recovery,Documentation Request,,
5,1024619926727,Exchange Request for Malfunctioning Dell XPS 13,"Dear Tech Online Store Customer Support,\n\n\n...","Dear <name>,\n\n\nThank you for reaching out a...",Change,Returns and Exchanges,medium,en,Tech Online Store,Returns and Exchanges,Product Replacement,Customer Service,Technical Support,Order Issue,,,,
6,1026487543175,Request for Server Administration Assistance,"Dear IT Services Customer Support, \n\n\nWe ar...",Subject: Re: Request for Server Administration...,Request,Product Support,medium,en,IT Services,Technical Support,IT Support,Performance Tuning,Problem Resolution,Technical Guidance,Service Recovery,,,
7,1027850274374,Immediate Attention Required: AWS Outage Concern,"Greetings IT Services Customer Support,\n\n\nI...","Hello <name>,\n\n\nWe appreciate you informing...",Incident,Technical Support,high,en,IT Services,Service Outage,Urgent Issue,IT Support,System Maintenance,Incident Report,Customer Service,,,


In [ ]:
df_en.shape[0] # no of rows in dataset df_en

339

In [ ]:
df_en.loc[df_en["priority"].eq("high"), :].shape[0] #high priority entities in the dataset

176

In [ ]:
df_en.loc[df_en["priority"].eq("high"), :].shape[0] / df_en.shape[0] *100 # percentage of high priority entities in the dataset

51.91740412979351

In [ ]:
# combine all tags columns from 1 to 9, and get value counts for most occuring words in this column after splitting and combining all lists

# Combine tags from tag_1 to tag_9
all_tags = []
for index, row in df_en.iterrows():
    tags = []
    for i in range(1, 4):  # Iterate through tag_1 to tag_3
        tag_value = row[f"tag_{i}"]
        if pd.notna(tag_value) and tag_value != '':
            tags.extend(str(tag_value).split())
    all_tags.extend(tags)

# Calculate value counts for the combined tags
from collections import Counter
tag_counts = Counter(all_tags)
print(tag_counts.most_common(100)) # Print the 10 most common words

[('Support', 447), ('Technical', 262), ('Issue', 215), ('Urgent', 133), ('Service', 105), ('Product', 104), ('IT', 98), ('Network', 51), ('Disruption', 48), ('Software', 47), ('Bug', 47), ('Failure', 35), ('Hardware', 34), ('Customer', 29), ('Problem', 26), ('Resolution', 26), ('Billing', 24), ('Inquiry', 24), ('Outage', 20), ('General', 16), ('Account', 16), ('Assistance', 16), ('Performance', 15), ('Tuning', 15), ('Guidance', 15), ('System', 14), ('Returns', 11), ('and', 11), ('Exchanges', 11), ('Request', 11), ('Maintenance', 10), ('Incident', 9), ('Report', 9), ('Sales', 8), ('Payment', 6), ('Processing', 6), ('Feature', 6), ('Notification', 5), ('Warranty', 5), ('Claim', 5), ('Crash', 5), ('Setup', 4), ('Refund', 4), ('Order', 4), ('Printer', 3), ('Login', 3), ('Replacement', 2), ('Wireless', 2), ('Services', 2), ('Cost', 2), ('Recovery', 2), ('Management', 2), ('Consulting', 1), ('Data', 1), ('Breach', 1), ('Feedback', 1), ('Password', 1), ('Reset', 1), ('Email', 1), ('Access', 1

In [ ]:
import re

def should_escalate(incoming_issue):
    if incoming_issue["priority"] == "high":
        # Combine all tags into one string
        tags_combined = " ".join([incoming_issue[f"tag_{i+1}"] for i in range(9)])

        # Keywords that trigger escalation
        keywords = ["Disruption","urgent","issue","refund", "Failure", "Outage", "Incident", "Crash", "Breach", "Critical"]

        # Check if any keyword matches in the combined tags using regular expressions for whole word matching
        for key in keywords:
            if re.search(r'\b' + re.escape(key.lower()) + r'\b', tags_combined.lower()):
                return True

    return False

In [ ]:
def adjust_priority_based_on_severity(incoming_issue):
    severity_keywords = ["Critical", "Emergency", "High"]
    tags_combined = " ".join([incoming_issue[f"tag_{i+1}"] for i in range(9)])

    # Check for severity keywords and adjust priority
    if any(kw.lower() in tags_combined.lower() for kw in severity_keywords):
        if incoming_issue["priority"] == "medium":
            incoming_issue["priority"] = "high"  # Promote priority for urgent keywords

    return incoming_issue

In [ ]:
def escalation_score(incoming_issue):
    tags_combined = " ".join([incoming_issue[f"tag_{i+1}"] for i in range(9)])
    score = sum(1 for key in ["Disruption", "Failure", "Outage", "Incident"] if key.lower() in tags_combined.lower())
    return score

In [ ]:
# Example: Test the escalation function with an incoming issue
incoming_issue = {
    "priority": "high",
    "tag_1": "Failure",
    "tag_2": "Crash",
    "tag_3": "Server",
    "tag_4": "",
    "tag_5": "",
    "tag_6": "",
    "tag_7": "",
    "tag_8": "",
    "tag_9": ""
}

# Check if the issue should be escalated
if should_escalate(incoming_issue):
    print("Escalate the issue")
else:
    print("Do not escalate the issue")

# Adjust priority based on severity (if needed)
incoming_issue = adjust_priority_based_on_severity(incoming_issue)
print(incoming_issue["priority"])

# Calculate escalation score
score = escalation_score(incoming_issue)
print(f"Escalation score: {score}")

Escalate the issue
high
Escalation score: 1


In [ ]:
# prompt: escalate issues for all of the rows

def escalate_all_rows(df):
    """Escalates issues for all rows in the DataFrame based on defined criteria."""

    escalated_rows = []
    for index, row in df.iterrows():
        issue = row.to_dict()
        if should_escalate(issue):
            issue["escalated"] = True  # Add an "escalated" column
            issue = adjust_priority_based_on_severity(issue)
            issue["escalation_score"] = escalation_score(issue)
            escalated_rows.append(issue)
        else:
            issue["escalated"] = False
            issue["escalation_score"] = escalation_score(issue)
            escalated_rows.append(issue)

    return pd.DataFrame(escalated_rows)

# Assuming 'df_en' is your DataFrame (from the previous code)
df_escalated = escalate_all_rows(df_en)

# Convert the DataFrame back to a list of lists for writing to Google Sheets
updated_data = df_escalated.values.tolist()

# Insert headers
header_row = df_escalated.columns.tolist()  # Update header
updated_data.insert(0, header_row)

# Update the Google Sheet with the updated data
try:
    worksheet.update("A1", updated_data)  # Use "A1" to overwrite the entire sheet
    print("Successfully updated Google Sheet with escalated issues.")
except Exception as e:
    print(f"Error updating Google Sheet: {e}")

<ipython-input-14-993812d2168d>:33: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update("A1", updated_data)  # Use "A1" to overwrite the entire sheet


Error updating Google Sheet: APIError: [401]: API keys are not supported by this API. Expected OAuth2 access token or other authentication credentials that assert a principal. See https://cloud.google.com/docs/authentication


In [ ]:
df = df_escalated
df.head()

,id,subject,body,answer,type,queue,priority,language,business_type,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9,escalated,escalation_score
0,1001352387736,Urgent: Critical impact on enterprise network ...,"Dear Customer Support Team, We are experiencin...",Subject: Re: Urgent: Critical impact on enterp...,Incident,Technical Support,high,en,IT Services,Service Outage,Network Issue,Urgent Issue,Technical Support,Problem Resolution,Critical Failure,System Crash,Service Disruption,,True,3
1,1004699418379,Intermittent Cursor Freezing Issue on Dell XPS,"Dear Customer Support,<br><br>I hope this mess...","Dear <name>,\n\n\nThank you for reaching out r...",Incident,Product Support,low,en,Tech Online Store,Technical Support,Product Support,Hardware Failure,Problem Resolution,Urgent Issue,Service Recovery,Documentation Request,,,False,1
2,1024619926727,Exchange Request for Malfunctioning Dell XPS 13,"Dear Tech Online Store Customer Support,\n\n\n...","Dear <name>,\n\n\nThank you for reaching out a...",Change,Returns and Exchanges,medium,en,Tech Online Store,Returns and Exchanges,Product Replacement,Customer Service,Technical Support,Order Issue,,,,,False,0
3,1026487543175,Request for Server Administration Assistance,"Dear IT Services Customer Support, \n\n\nWe ar...",Subject: Re: Request for Server Administration...,Request,Product Support,medium,en,IT Services,Technical Support,IT Support,Performance Tuning,Problem Resolution,Technical Guidance,Service Recovery,,,,False,0
4,1027850274374,Immediate Attention Required: AWS Outage Concern,"Greetings IT Services Customer Support,\n\n\nI...","Hello <name>,\n\n\nWe appreciate you informing...",Incident,Technical Support,high,en,IT Services,Service Outage,Urgent Issue,IT Support,System Maintenance,Incident Report,Customer Service,,,,True,2


In [ ]:
import requests
import json

def escalate_issue(issue_data, api_endpoint):
    """
    Sends a POST request to the /check_escalate API endpoint to escalate an issue.

    Args:
        issue_data (dict): A dictionary containing the issue data.
        api_endpoint (str): The URL of the /check_escalate API endpoint.

    Returns:
        dict: The JSON response from the API, or None if there was an error.
    """
    try:
        headers = {'Content-type': 'application/json'}  # Set the content type to JSON
        response = requests.post(api_endpoint, data=json.dumps(issue_data), headers=headers)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

        return response.json()

    except requests.exceptions.RequestException as e:
        print(f"Error escalating issue: {e}")
        return None

# Example Usage (replace with your actual API endpoint and issue data)
api_url = "your_api_endpoint/check_escalate"

# Assuming 'df_escalated' is your DataFrame with the 'escalated' column (from the previous code)

for index, row in df_escalated.iterrows():
    if row['escalated']:
        issue_data = row.to_dict()  # Convert the row to a dictionary
        escalation_response = escalate_issue(issue_data, api_url)

        if escalation_response:
            print(f"Escalation successful for issue {index}: {escalation_response}")
        else:
            print(f"Escalation failed for issue {index}")

Error escalating issue: Invalid URL 'your_api_endpoint/check_escalate': No scheme supplied. Perhaps you meant https://your_api_endpoint/check_escalate?
Escalation failed for issue 0
Error escalating issue: Invalid URL 'your_api_endpoint/check_escalate': No scheme supplied. Perhaps you meant https://your_api_endpoint/check_escalate?
Escalation failed for issue 4
Error escalating issue: Invalid URL 'your_api_endpoint/check_escalate': No scheme supplied. Perhaps you meant https://your_api_endpoint/check_escalate?
Escalation failed for issue 5
Error escalating issue: Invalid URL 'your_api_endpoint/check_escalate': No scheme supplied. Perhaps you meant https://your_api_endpoint/check_escalate?
Escalation failed for issue 7
Error escalating issue: Invalid URL 'your_api_endpoint/check_escalate': No scheme supplied. Perhaps you meant https://your_api_endpoint/check_escalate?
Escalation failed for issue 10
Error escalating issue: Invalid URL 'your_api_endpoint/check_escalate': No scheme supplie